In [ ]:
seconds_on_domain_per_session = await get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
interventions_active_for_domain_and_session = await get_collection_for_user(user_id, 'synced:interventions_active_for_domain_and_session')
